<a href="https://colab.research.google.com/github/aquila00/MovieIt/blob/main/MovieIt_item.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de Filtragem colaborativa - User Based

##Database:

This dataset (ml-25m) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service.

It contains:

* 25.000.095 ratings;
*  1.093.360 tag applications;
*    62.423 movies.

Ratings: 0.5 stars - 5.0 stars

These data were created by 162.541 users between January 09, 1995 and November 21, 2019. 

This dataset was generated on November 21, 2019.

#Pré processamento

##Bibliotecas

In [ ]:
# Gerais
import pandas as pd
import numpy as np
import math

# Para separação de treino-teste
from sklearn.model_selection import train_test_split

# Para o RMSE
from sklearn.metrics import mean_squared_error

# Plot
import matplotlib.pyplot as plt
from pylab import title, xlabel, ylabel

# Google drive / Colab
from google.colab import drive

In [ ]:
# Acessando os arquivos no GoogleDrive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Database

### Ratings

In [ ]:
# 1 milhão
path = '/content/gdrive/MyDrive/movieit/ml-1m'
db_rating = pd.read_csv(f'{path}/ratings.dat', names=['userid', 'movieid', 'rating', 'timestamp'], usecols=['userid', 'movieid', 'rating'], skiprows=1, header=None, sep='::')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


### Gêneros

In [ ]:
# gêneros
db_movie = pd.read_csv(f'{path}/movies.dat', names=['movieid', 'movietitle', 'genres'], skiprows=1, header=None, sep='::')

In [ ]:
db_movie

,movieid,movietitle,genres
0,2,Jumanji (1995),Adventure|Children's|Fantasy
1,3,Grumpier Old Men (1995),Comedy|Romance
2,4,Waiting to Exhale (1995),Comedy|Drama
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
3877,3948,Meet the Parents (2000),Comedy
3878,3949,Requiem for a Dream (2000),Drama
3879,3950,Tigerland (2000),Drama
3880,3951,Two Family House (2000),Drama


##Exploratória

In [ ]:
db_rating

,userid,movieid,rating
0,1,661,3
1,1,914,3
2,1,3408,4
3,1,2355,5
4,1,1197,3
...,...,...,...
1000203,6040,1091,1
1000204,6040,1094,5
1000205,6040,562,5
1000206,6040,1096,4


In [ ]:
n_ratings = len(db_rating)
n_ratings: int = len(db_rating)
n_users = len(db_rating['userid'].unique())
n_movies_avaliados = len(db_rating['movieid'].unique())
esparsidade = round(1.0 - n_ratings/float(n_users * n_movies_avaliados), 3)

print(f'Total de ratings: {n_ratings}\n'\
      f'Total de usuários únicos: {n_users}\n'\
      f'Total de filmes: {n_movies_avaliados}\n'\
      f'Média de ratings/user: {round(n_ratings/n_users, 2)}\n'\
      f'Esparsidade da base é de: {(esparsidade*100)}%'\
      f'\n')

print(f'{db_rating.info()}')

Total de ratings: 1000208
Total de usuários únicos: 6040
Total de filmes: 3706
Média de ratings/user: 165.6
Esparsidade da base é de: 95.5%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000208 entries, 0 to 1000207
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   userid   1000208 non-null  int64
 1   movieid  1000208 non-null  int64
 2   rating   1000208 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB
None


## Criando os conjuntos de treino e teste

**Considerações:**

Os conjuntos de treino e de teste foram criados pelos ids dos usuários uma vez que o conjunto da votação de um usuário compõe a integridade de suas preferências. 

A separação dos dados sem considerar esta integridade geraria problemas de particionamento do perfil do usuário alterando o resultado da similaridade.

In [ ]:
# matriz dos ids únicos dos usuários para a refereência do particionamento treino e teste
print(db_rating.userid.unique())

[   1    2    3 ... 6038 6039 6040]


In [ ]:
# Separando o conjunto de treino e de teste, deixando 10% dos usuários para futuros testes
X_train, X_test = train_test_split(db_rating.userid.unique(), test_size=0.2, random_state=700)

In [ ]:
# retornando os valores de filmes e de ratings para completar a matriz de treino
X_train = db_rating.loc[db_rating['userid'].isin(X_train)]

# retornando os valores de filmes e de ratings para completar a matriz de teste
X_test = db_rating.loc[db_rating['userid'].isin(X_test)]

### X_train - Exploratória

In [ ]:
X_train.head()

,userid,movieid,rating
52,2,1357,5
53,2,3068,4
54,2,1537,4
55,2,647,3
56,2,2194,4


In [ ]:
X_train.count()

userid     807160
movieid    807160
rating     807160
dtype: int64

In [ ]:
X_train.head()

,userid,movieid,rating
52,2,1357,5
53,2,3068,4
54,2,1537,4
55,2,647,3
56,2,2194,4


In [ ]:
n_ratings = len(X_train)
n_users = len(X_train['userid'].unique())
n_movies_avaliados = len(X_train['movieid'].unique())
esparsidade = round(1.0 -n_ratings/float(n_users * n_movies_avaliados),3)

print(f'Total de ratings: {n_ratings}\n'\
      f'Total de usuários únicos: {n_users}\n'\
      f'Total de filmes: {n_movies_avaliados}\n'\
      f'Média de ratings/user: {round(n_ratings/n_users, 2)}\n'\
      f'Esparsidade da base é de: {(esparsidade*100)}%'\
      f'\n')

print(f'{db_rating.info()}')

Total de ratings: 807160
Total de usuários únicos: 4832
Total de filmes: 3691
Média de ratings/user: 167.04
Esparsidade da base é de: 95.5%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000208 entries, 0 to 1000207
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   userid   1000208 non-null  int64
 1   movieid  1000208 non-null  int64
 2   rating   1000208 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB
None


### X_test - Exploratória

In [ ]:
X_test.head()

,userid,movieid,rating
0,1,661,3
1,1,914,3
2,1,3408,4
3,1,2355,5
4,1,1197,3


In [ ]:
X_test.count()

userid     193048
movieid    193048
rating     193048
dtype: int64

In [ ]:
X_test.head()

,userid,movieid,rating
0,1,661,3
1,1,914,3
2,1,3408,4
3,1,2355,5
4,1,1197,3


In [ ]:
n_ratings = len(X_test)
n_users = len(X_test['userid'].unique())
n_movies_avaliados = len(X_test['movieid'].unique())
esparsidade = round(1.0 -n_ratings/float(n_users * n_movies_avaliados),3)

print(f'Total de ratings: {n_ratings}\n'\
      f'Total de usuários únicos: {n_users}\n'\
      f'Total de filmes: {n_movies_avaliados}\n'\
      f'Média de ratings/user: {round(n_ratings/n_users, 2)}\n'\
      f'Esparsidade da base é de: {(esparsidade*100)}%'\
      f'\n')

print(f'{db_rating.info()}')

Total de ratings: 193048
Total de usuários únicos: 1208
Total de filmes: 3446
Média de ratings/user: 159.81
Esparsidade da base é de: 95.39999999999999%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000208 entries, 0 to 1000207
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   userid   1000208 non-null  int64
 1   movieid  1000208 non-null  int64
 2   rating   1000208 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB
None


# Recomendando filme para usuário único

## Filtragem da base

### Funções

In [ ]:
def instanciando_usuario_unico(id_userunico):
    """O "uservip" é instanciado a partir de um um usuário real do conjunto X_test visando a otimização de tempo para a criação de um novo usuário."""

    # Uma amostra do X_test para otimização de tempo, utilizando o id informado na variável id_arbitrario
    userunico = X_test.loc[X_test['userid'] == id_userunico]
    userunico = userunico.loc[userunico['rating'] == 5]

    userunico = userunico.iloc[:2]

    return userunico

In [ ]:
def criar_base_para_correlacao(X_treino, usuario):
    """Seleciona todos os usuários da base de treino que tenham assistido pelo menos um filme em comum com o "usuario" e o próprio usuario, pivotados"""

    base = X_treino.loc[X_treino['movieid'].isin(usuario['movieid']), 'userid']
    base = X_treino.loc[X_treino['userid'].isin(base)]
    # Adiciona o usuario à tabela para que o número de colunas seja igual ao da base e para, futuramente, fazer a correlação
    base = pd.concat([base, usuario], ignore_index=True, axis='index')
    base_pivotada = base.pivot_table(index='userid', columns='movieid', values='rating')

    return base_pivotada

In [ ]:
def calculando_correlacao(base, id_filme, metodo='pearson'):
    """Calculando pearson por default"""

    uservip_corr = base.corrwith(base[id_filme])
    uservip_corr.drop(index=id_filme, inplace=True)
    uservip_corr = uservip_corr[uservip_corr>0]
    similaridades = uservip_corr.to_list()
    indexes = uservip_corr.index.to_list()    
    dicionario = dict(zip(indexes, similaridades))
   
    return dicionario

In [ ]:
def recomendacoes(base_treino, userid=1, metodo_de_calculo='pearson', rmse=False, rec_por_filmes=3):

    filmes_recomendados = []
    user = instanciando_usuario_unico(userid)

    db_corr = criar_base_para_correlacao(base_treino, user)

    for filme in user.movieid:
        # Calcula a correlação existente entre o filme escolhido e os filmes filtrados pelo usuário
        correlacao = calculando_correlacao(db_corr, filme, metodo_de_calculo)
        
        rec = [[i, correlacao[i]] for i in correlacao if i not in user.movieid]
        rec.sort(key = lambda x: x[1])
        rec = rec[:rec_por_filmes]
    
        if rmse == False:
            rec = [linha[0] for linha in rec]
     
        filmes_recomendados.append([filme, rec])

    return filmes_recomendados

### Recomendando

In [ ]:
#ordena os filmes por ordem ascendente de notas e printa a nota na tela
userid = np.random.choice(X_test.userid.unique(), 1)
filmes_recomendados = recomendacoes(X_train, rec_por_filmes=10)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [ ]:
filmes_recomendados

[[2355, [1194, 2912, 3617, 288, 1131, 1665, 3182, 3620, 3375, 605]],
 [1287, [246, 1367, 1033, 3152, 2136, 345, 216, 2091, 1320, 3416]]]

In [ ]:
for i in range(len(filmes_recomendados)):
    print(f"Já que você assistiu o filme {db_movie.loc[db_movie['movieid'] == filmes_recomendados[i][0], 'movietitle'].iloc[0]}, te recomendamos os filmes:")
    print("")

    for j in range(len(filmes_recomendados[i][1])):
        print(f"\t{j+1} ─ {db_movie.loc[db_movie['movieid'] == filmes_recomendados[i][1][j], 'movietitle'].iloc[0]}")
    
    print("")
    print("")

Já que você assistiu o filme Bug's Life, A (1998), te recomendamos os filmes:

	1 ─ Up in Smoke (1978)
	2 ─ Limey, The (1999)
	3 ─ Road Trip (2000)
	4 ─ Natural Born Killers (1994)
	5 ─ Jean de Florette (1986)
	6 ─ Bean (1997)
	7 ─ Mr. Death: The Rise and Fall of Fred A. Leuchter Jr. (1999)
	8 ─ Myth of Fingerprints, The (1997)
	9 ─ Destination Moon (1950)
	10 ─ One Fine Day (1996)


Já que você assistiu o filme Ben-Hur (1959), te recomendamos os filmes:

	1 ─ Hoop Dreams (1994)
	2 ─ 101 Dalmatians (1996)
	3 ─ Fox and the Hound, The (1981)
	4 ─ Last Picture Show, The (1971)
	5 ─ Nutty Professor, The (1963)
	6 ─ Adventures of Priscilla, Queen of the Desert, The (1994)
	7 ─ Billy Madison (1995)
	8 ─ Return from Witch Mountain (1978)
	9 ─ Alien³ (1992)
	10 ─ Trial, The (Le Procès) (1963)




# Medidas de avaliação

In [ ]:
users = np.random.choice(X_test.userid.unique(), 3, replace=False)

In [ ]:
for user_temp in users:
    similaridades_original = recomendacoes(X_test, userid=user_temp, rmse=True)

    X_test_temp = X_test.drop(np.random.choice(X_test.userid.unique(), math.floor(len(np.random.choice(X_test.userid.unique()) * 0.2), replace=False)))

    similaridades_predito = recomendacoes(X_test, userid=user_temp, rmse=True)

    user_original = user_original.sort_values(by=['movieid'])
    user_predito = user_predito.sort_values(by=['movieid'])

    plt.plot(user_original.movieid, user_original.rating)
    plt.plot(user_predito.movieid, user_predito.rating)     
    
    title(f"Notas para o usuário {user_temp}")
    xlabel("Filme")
    ylabel("Notas")
    plt.legend(['original', 'predito'])

    rms = mean_squared_error(user_original.rating, user_predito.rating, squared=False)

    print('RMSE:')
    print(rms)
    plt.show()

TypeError: ignored